Preprocessing

In [1]:
from requests import get

def download_file(url, file_name):
    with open('gzip.zip', "wb") as file:
        response = get(url)
        file.write(response.content)

download_file('http://www.itl.nist.gov/iaui/vip/cs_links/EMNIST/gzip.zip', 'gzip.zip')

#Extract Files 



def extract(gzip_path):
    zip_ref = zipfile.ZipFile(gzip_path, 'r')
    zip_ref.extractall('datasets')
    zip_ref.close()
    for file in glob.glob(r'datasets/gzip/emnist-byclass*'):
        shutil.move(file, 'datasets/')
    shutil.rmtree('datasets/gzip')
    os.remove('gzip.zip')
extract('gzip.zip')



%matplotlib inline
import matplotlib.pyplot as plt


def read_emnist(images_path: str, labels_path: str):
    with gzip.open(labels_path, 'rb') as labelsFile:
        labels = np.frombuffer(labelsFile.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path,'rb') as imagesFile:
        length = len(labels)
        # Load flat 28x28 px images (784 px), and convert them to 28x28 px
        features = np.frombuffer(imagesFile.read(), dtype=np.uint8, offset=16) \
                        .reshape(length, 784) \
                        .reshape(length, 28, 28, 1)
        
    return features, labels

    

NameError: name 'zipfile' is not defined

In [ ]:
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

import zipfile
import shutil
import glob
import os
import gzip
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
import tensorflow as tf

In [ ]:

def read_emnist(images_path: str, labels_path: str):
    with gzip.open(labels_path, 'rb') as labelsFile:
        labels = np.frombuffer(labelsFile.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path,'rb') as imagesFile:
        length = len(labels)
        # Load flat 28x28 px images (784 px), and convert them to 28x28 px
        features = np.frombuffer(imagesFile.read(), dtype=np.uint8, offset=16) \
                        .reshape(length, 784) \
                        .reshape(length, 28, 28, 1)
        
    return features, labels

In [ ]:
train = {}
test = {}

train['features'], train['labels'] = read_emnist('datasets/emnist-byclass-train-images-idx3-ubyte.gz', 'datasets/emnist-byclass-train-labels-idx1-ubyte.gz')
test['features'], test['labels'] = read_emnist('datasets/emnist-byclass-test-images-idx3-ubyte.gz', 'datasets/emnist-byclass-test-labels-idx1-ubyte.gz')

    

In [ ]:
def rotate(img):
    # Used to rotate images (for some reason they are transposed on read-in)
    flipped = np.fliplr(img)
    return np.rot90(flipped)

In [ ]:
# fix the orientation of the images
train['features'] = np.array(list(map(rotate, train['features'])))

In [ ]:
test['features'] = np.array(list(map(rotate, test['features'])))

In [ ]:
# Get the ascii label
label_map = {}
def load_label_map(map_path):
    with open(map_path, 'r') as map_file:
        for line in map_file:
            mapping = line.split()
            label_map[int(mapping[0])] = chr(int(mapping[1]))
load_label_map('datasets/emnist-byclass-mapping.txt')


### Explore the data

In [ ]:
def display_image(mode, position):
    if mode == 'train':
        image = train['features'][position].squeeze()
        plt.title('%d. Label: %s' % (position, label_map[train['labels'][position]]))
    else:
        image = test['features'][position].squeeze()
        plt.title('%d. Label: %s' % (position, label_map[test['labels'][position]]))
    plt.imshow(image, cmap=plt.cm.gray_r)

In [ ]:
display_image('train', 1210)

In [ ]:
display_image('test', 1001)

### Training data stats

In [ ]:
train_labels_count = np.unique(train['labels'], return_counts=True)
dataframe_train_labels = pd.DataFrame({'Label':train_labels_count[0], 'Count':train_labels_count[1]})
dataframe_train_labels

### Create a validation set

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
validation = {}
train['features'], validation['features'], train['labels'], validation['labels'] = train_test_split(train['features'], train['labels'], test_size=0.2, random_state=0)

In [ ]:
print('# of training images:', train['features'].shape[0])
print('# of validation images:', validation['features'].shape[0])

### Prepare our input features (padding 28x28 to 32x32)

In [ ]:
# Pad images with 0s
train['features']      = np.pad(train['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
validation['features'] = np.pad(validation['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
test['features']       = np.pad(test['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(train['features'][0].shape))



```
# This is formatted as code
```

## Build the Custom model

In [ ]:


# input shape is 32,32,1
# The custom model is a Deep CNN with 2 conv layers and 2 hidden layers 

def build_model():
    model = Sequential()
   
    # 2 Convolutional Layers 

    #The first 
    model.add(Conv2D(filters=6, kernel_size=(3,3), activation='relu', input_shape= (32,32,1)))
    model.add(AveragePooling2D()) # Average Pooling for subsampling 
    
    # Second 
    model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
    model.add(AveragePooling2D()) # Average Pooling for subsampling 
    model.add(Flatten())
    
    # 2 Hidden Layers 
    model.add(Dense(units=120, activation='relu'))
    model.add(Dense(units=84, activation='relu'))

    #Output Layer
    model.add(Dense(units=62, activation='softmax'))
    
    # model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    #print(model.summary())
    model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    print(model.summary())
    #return model
    return model


In [ ]:
model = build_model()

### Train

In [ ]:
EPOCHS = 3
BATCH_SIZE = 128

In [ ]:
X_train, y_train = train['features'], to_categorical(train['labels'])
X_validation, y_validation = validation['features'], to_categorical(validation['labels'])

train_generator = ImageDataGenerator().flow(X_train, y_train, batch_size=BATCH_SIZE)
validation_generator = ImageDataGenerator().flow(X_validation, y_validation, batch_size=BATCH_SIZE)

In [ ]:
print('# of training images:', train['features'].shape[0])
print('# of validation images:', validation['features'].shape[0])

steps_per_epoch = X_train.shape[0]//BATCH_SIZE
validation_steps = X_validation.shape[0]//BATCH_SIZE

model.fit(X_train,y_train,epochs=3) 


### Evaluate the trained model
Evaluate on the test set. The model has not seen any of these examples during training.

In [ ]:
score = model.evaluate(test['features'], to_categorical(test['labels']))
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Use the model for prediction

In [ ]:
def test_the_model(position):
    out = model.predict(np.reshape(test['features'][position], (1,32,32,1)))
    image = test['features'][position].squeeze()
    plt.title('I: %d. True Label: %s, Predicted: %s' % (position, label_map[test['labels'][position]], label_map[np.argmax(out)]))
    plt.imshow(image, cmap=plt.cm.gray_r)

In [ ]:
test_the_model(100)

### Save the trained model


In [ ]:
from keras.models import save_model

In [ ]:
# Offload model to file
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
save_model(model, 'model.h5')

### Load the trained model

In [ ]:
from keras.models import model_from_yaml
def load_model():
    ''' Load model from .yaml and the weights from .h5
        Arguments:
            bin_dir: The directory of the bin (normally bin/)
        Returns:
            Loaded model from file
    '''

    # load YAML and create model
    yaml_file = open('model.yaml', 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    model = model_from_yaml(loaded_model_yaml)

    # load weights into new model
    model.load_weights('model.h5')
    return model

trained_model = load_model()

### Use the trained model for prediction

In [ ]:
def test_the_trained_model(position):
    out = trained_model.predict(np.reshape(test['features'][position], (1,32,32,1)))
    image = test['features'][position].squeeze()
    plt.title('I: %d. True Label: %s, Predicted: %s' % (position, label_map[test['labels'][position]], label_map[np.argmax(out)]))
    plt.imshow(image, cmap=plt.cm.gray_r)

In [ ]:
test_the_model(1002)

#Estimator

In [ ]:
my_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model,
    model_dir='models/estimator-for-XOR/')

In [ ]:
def train_input_fn(x_train, y_train, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(
        ({'input-features':x_train}, y_train.reshape(-1, 62)))

    # Shuffle, repeat, and batch the examples.
    return dataset.shuffle(100).repeat().batch(batch_size)

def eval_input_fn(x_test, y_test=None, batch_size = 128):
    if y_test is None:
        dataset = tf.data.Dataset.from_tensor_slices(
            {'input-features':x_test})
    else:
        dataset = tf.data.Dataset.from_tensor_slices(
            ({'input-features':x_test}, y_test.reshape(-1, -1)))


    # Shuffle, repeat, and batch the examples.
    return dataset.batch(batch_size)


In [ ]:
X_train, y_train = train['features'], to_categorical(train['labels'])
X_validation, y_validation = validation['features'], to_categorical(validation['labels'])

steps_per_epoch = X_train.shape[0]//BATCH_SIZE

EPOCHS = 3
BATCH_SIZE = 128
steps_per_epoch = np.ceil(len(X_train)/BATCH_SIZE)

train_input_fn(X_train, y_train, BATCH_SIZE)
input_fn=lambda: train_input_fn(X_train, y_train, BATCH_SIZE)

my_estimator.train(
    input_fn=input_fn,
    steps=EPOCHS*BATCH_SIZE)

In [ ]:
34617390/558345